In [3]:
import sys
sys.path.append("../")
sys.path.append("../data_retrieval/data")
from data_retrieval.data import split_data
from torchvision.models import resnet101, ResNet101_Weights
from torchvision.datasets import ImageFolder
import torch
import json
from sklearn.metrics import confusion_matrix

In [4]:
labasi_data = ImageFolder('../data_retrieval/labasi_data/data/',ResNet101_Weights.IMAGENET1K_V2.transforms())

In [5]:
model = resnet101()

In [6]:
model.load_state_dict(torch.load('../models/models_weights/resnet101.pth'))

<All keys matched successfully>

In [7]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
test_loader = torch.utils.data.DataLoader(labasi_data,
                                             batch_size=16, shuffle=True,
                                             num_workers=4)

In [9]:
idx_to_class = {v: k for k, v in labasi_data.class_to_idx.items()}

In [10]:
print(idx_to_class)

{0: 'E', 1: 'GAR', 2: 'KA', 3: 'KI', 4: 'MEŠ', 5: 'NI', 6: 'RU', 7: 'TA', 8: 'TI', 9: 'U₂', 10: 'ŠU'}


In [11]:
f = open("../data_retrieval/data/mapping")
mapping = json.load(f)

In [12]:
correct_pred = {classname: 0 for classname in idx_to_class}
total_pred = {classname: 0 for classname in idx_to_class}
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if idx_to_class[int(label)] + "_Neo-Babylonian" == mapping[str(int(prediction))]:
                correct_pred[int(label)] += 1
            total_pred[int(label)] += 1


In [13]:
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {idx_to_class[classname]:5s} is {accuracy:.1f} %')

Accuracy for class: E     is 80.7 %
Accuracy for class: GAR   is 96.8 %
Accuracy for class: KA    is 74.4 %
Accuracy for class: KI    is 76.6 %
Accuracy for class: MEŠ   is 83.6 %
Accuracy for class: NI    is 90.4 %
Accuracy for class: RU    is 86.0 %
Accuracy for class: TA    is 88.4 %
Accuracy for class: TI    is 78.4 %
Accuracy for class: U₂    is 88.3 %
Accuracy for class: ŠU    is 64.9 %


In [15]:
print("Overall accuracy: ",sum(correct_pred.values()) / sum(total_pred.values()))

Overall accuracy:  0.824045407636739
